In [126]:
from polygon import RESTClient
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import json
import datetime
import numpy as np
load_dotenv()

portfolio_data = pd.DataFrame()


## Importing Polyon Stock Data

In [127]:
def load_portfolio_data(ticker : str, name: str, type : str, portfolio_data: pd.DataFrame,):
    client = RESTClient(os.environ['polygon_api_key'])
    aggs = client.get_aggs(
        ticker,
        1,
        "day",
        "2023-01-01",
        str(datetime.date.today()),
        limit=5000
    )
    df = pd.DataFrame(aggs).assign(ticker=ticker, name=name, type=type)
    return pd.concat([portfolio_data, df])

In [128]:
portfolio_data = load_portfolio_data(
    ticker="TSM",
    name="Taiwan Semiconductor Mfg. Co. Ltd.",
    type="stock",
    portfolio_data=portfolio_data
)

In [129]:
portfolio_data = load_portfolio_data(
    ticker="AMZN",
    name="Amazon.com Inc",
    type="stock",
    portfolio_data=portfolio_data
)

In [130]:
portfolio_data = load_portfolio_data(
    ticker="NVDA",
    name="NVIDIA Corp",
    type="stock",
    portfolio_data=portfolio_data
)

In [131]:
portfolio_data = load_portfolio_data(
    ticker="AXP",
    name="American Express Company",
    type="stock",
    portfolio_data=portfolio_data
)

In [132]:
portfolio_data = load_portfolio_data(
    ticker="CELH",
    name="Celsius Holdings, Inc.",
    type="stock",
    portfolio_data=portfolio_data
)

In [133]:
portfolio_data = load_portfolio_data(
    ticker="C:USDJPY",
    name="USD/JPY Foreign Exchange",
    type="forex",
    portfolio_data=portfolio_data
)

In [134]:
portfolio_data = load_portfolio_data(
    ticker="SPY",
    name="SPDR S&P 500 ETF Trust",
    type="etf",
    portfolio_data=portfolio_data
)

In [244]:
client = RESTClient(os.environ['polygon_api_key'])
NDX_data = pd.DataFrame(client.get_aggs(
    'I:NDX',
    1,
    "day",
    "2023-01-01",
    str(datetime.date.today()),
    limit=5000
))

In [135]:
# Check to make sure all stock data is loaded in
portfolio_data["ticker"].unique()

array(['TSM', 'AMZN', 'NVDA', 'AXP', 'CELH', 'C:USDJPY', 'SPY'],
      dtype=object)

## Data Cleaning

In [136]:
# resetting the index
portfolio_data.reset_index(drop=True, inplace=True)

In [137]:
# dropping the otc column
portfolio_data.drop(columns=["otc"], inplace=True)

In [138]:
# converting epoch times to dates
portfolio_data["timestamp"] = pd.to_datetime(portfolio_data["timestamp"], unit="ms").dt.date
portfolio_data = portfolio_data.rename(columns={'timestamp': 'date'})
portfolio_data["date"] = pd.to_datetime(portfolio_data["date"])


In [139]:
# formatting numbers
portfolio_data = portfolio_data.round(2)
portfolio_data["volume"] = portfolio_data["volume"].astype(int)

In [245]:
# cleaning index data
NDX_data.drop(columns=["volume","vwap","transactions","otc"], inplace=True)
NDX_data = NDX_data.rename(columns={'timestamp' : 'date'})
NDX_data['date'] = pd.to_datetime(NDX_data['date'], unit='ms').dt.date
NDX_data['date'] = pd.to_datetime(NDX_data['date'])
NDX_data = NDX_data.round(2)

NDX_data

,open,high,low,close,date
0,12085.67,12156.21,12006.04,12066.27,2023-02-22
1,11979.80,12018.32,11900.84,11969.65,2023-02-24
2,12106.79,12159.64,12034.61,12057.79,2023-02-27
3,12041.75,12146.52,12021.32,12042.12,2023-02-28
4,12026.72,12054.48,11906.58,11937.48,2023-03-01
...,...,...,...,...,...
344,20224.13,20406.99,20201.50,20391.97,2024-07-05
345,20393.89,20455.38,20363.37,20439.54,2024-07-08
346,20504.17,20543.90,20395.57,20453.02,2024-07-09
347,20533.27,20690.97,20479.94,20675.38,2024-07-10


In [246]:
NDX_data.describe()

,open,high,low,close,date
count,349.000000,349.000000,349.000000,349.000000,349
mean,15924.988911,16023.052980,15831.376476,15935.867822,2023-10-31 21:47:57.936962816
min,11752.100000,11908.390000,11695.410000,11830.280000,2023-02-22 00:00:00
25%,14642.080000,14774.160000,14557.830000,14694.240000,2023-06-29 00:00:00
50%,15561.150000,15618.850000,15429.360000,15508.240000,2023-10-30 00:00:00
75%,17815.320000,17864.160000,17676.350000,17783.170000,2024-03-06 00:00:00
max,20661.410000,20690.970000,20479.940000,20675.380000,2024-07-11 00:00:00
std,2102.702704,2101.548954,2096.389267,2102.681396,NaN


## Outlier Cleaning

In [140]:
# Checking for outliers in transactions and volume
portfolio_data.loc[portfolio_data["ticker"] == "C:USDJPY"].describe()

,open,high,low,close,volume,vwap,date,transactions
count,499.000000,499.000000,499.000000,499.000000,499.000000,499.000000,499,499.000000
mean,144.369739,144.882766,143.845511,144.437735,152266.703407,144.411463,2023-09-30 19:51:49.418837504,152266.703407
min,127.350000,127.880000,127.350000,127.870000,1.000000,127.880000,2023-01-01 00:00:00,1.000000
25%,136.595000,137.505000,136.285000,136.970000,109867.000000,136.805000,2023-05-13 12:00:00,109867.000000
50%,145.710000,146.220000,144.940000,145.730000,180092.000000,145.570000,2023-09-27 00:00:00,180092.000000
75%,150.095000,150.560000,149.800000,150.135000,213513.000000,150.105000,2024-02-17 00:00:00,213513.000000
max,161.580000,162.000000,161.300000,161.590000,357136.000000,161.590000,2024-07-11 00:00:00,357136.000000
std,8.498044,8.437833,8.542451,8.499978,86947.436292,8.487131,NaN,86947.436292


In [141]:
# Removing all weekend days
portfolio_data.drop(index=portfolio_data[portfolio_data["date"].dt.day_of_week>4].index, inplace=True)

# Removing all days the stock market is not open
no_holidays = portfolio_data['date'].value_counts() == 7
portfolio_data = portfolio_data[portfolio_data['date'].isin(no_holidays[no_holidays].index)]


## Aggregations

Calculating how many shares of what to buy (buying based off vwap and partial shares are allowed)
- 40% SPY
- 10% USDJPY
- 10% AMZN
- 10% TSM
- 10% CELH
- 10% AXP
- 10% NVDA

In [142]:
SPY_shares = 40000/portfolio_data[portfolio_data["ticker"] == "SPY"]["vwap"].iat[0]
USDJPY_shares = 10000/portfolio_data[portfolio_data["ticker"]=="C:USDJPY"]["vwap"].iat[0]
AMZN_shares = 10000/portfolio_data[portfolio_data["ticker"]=="AMZN"]["vwap"].iat[0]
TSM_shares = 10000/portfolio_data[portfolio_data["ticker"]=="TSM"]["vwap"].iat[0]
CELH_shares = 10000/portfolio_data[portfolio_data["ticker"]=="CELH"]["vwap"].iat[0]
AXP_shares = 10000/portfolio_data[portfolio_data["ticker"]=="AXP"]["vwap"].iat[0]
NVDA_shares = 10000/portfolio_data[portfolio_data["ticker"]=="NVDA"]["vwap"].iat[0]

Creating dataframes for each investment

In [143]:
cols = ['date','open','high','low','close','volume','vwap','transactions']
SPY_etf_data = portfolio_data[portfolio_data["ticker"] == "SPY"][cols].reset_index(drop=True)
USDJPY_forex_data = portfolio_data[portfolio_data["ticker"] == "C:USDJPY"][cols].reset_index(drop=True)
AMZN_stock_data = portfolio_data[portfolio_data["ticker"] == "AMZN"][cols].reset_index(drop=True)
TSM_stock_data = portfolio_data[portfolio_data["ticker"] == "TSM"][cols].reset_index(drop=True)
AXP_stock_data = portfolio_data[portfolio_data["ticker"] == "AXP"][cols].reset_index(drop=True)
CELH_stock_data = portfolio_data[portfolio_data["ticker"] == "CELH"][cols].reset_index(drop=True)
NVDA_stock_data = portfolio_data[portfolio_data["ticker"] == "NVDA"][cols].reset_index(drop=True)

In [144]:

portfolio = pd.concat([
    portfolio_data[portfolio_data["ticker"] == "SPY"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "C:USDJPY"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "AMZN"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "TSM"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "AXP"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "CELH"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "NVDA"][['ticker','name','type']].head(1)
]).reset_index(drop=True)

portfolio

,ticker,name,type
0,SPY,SPDR S&P 500 ETF Trust,etf
1,C:USDJPY,USD/JPY Foreign Exchange,forex
2,AMZN,Amazon.com Inc,stock
3,TSM,Taiwan Semiconductor Mfg. Co. Ltd.,stock
4,AXP,American Express Company,stock
5,CELH,"Celsius Holdings, Inc.",stock
6,NVDA,NVIDIA Corp,stock


Performing Stock/ETF/Forex aggregations

In [268]:
rf = 0.0546

SPY_etf_data['value'] = (SPY_etf_data['vwap'] * SPY_shares).round(2)
SPY_etf_data['cumulative return'] = (SPY_etf_data['value'] - SPY_etf_data['value'].iat[0])/SPY_etf_data['value'].iat[0]
SPY_etf_data['volatility'] = SPY_etf_data['vwap'].rolling(len(SPY_etf_data),min_periods=1).std()
SPY_etf_data['sharpe'] = (SPY_etf_data['cumulative return'] - rf)/(SPY_etf_data['cumulative return'].rolling(len(SPY_etf_data),min_periods=1).std())
SPY_etf_data.fillna(0, inplace=True)

AMZN_stock_data['value'] = (AMZN_stock_data['vwap'] * AMZN_shares).round(2)
AMZN_stock_data['cumulative return'] = (AMZN_stock_data['value'] - AMZN_stock_data['value'].iat[0])/AMZN_stock_data['value'].iat[0]
AMZN_stock_data['volatility'] = AMZN_stock_data['vwap'].rolling(len(AMZN_stock_data),min_periods=1).std()
AMZN_stock_data['sharpe'] = (AMZN_stock_data['cumulative return'] - rf)/(AMZN_stock_data['cumulative return'].rolling(len(AMZN_stock_data),min_periods=1).std())
AMZN_stock_data.fillna(0, inplace=True)

TSM_stock_data['value'] = (TSM_stock_data['vwap'] * TSM_shares).round(2)
TSM_stock_data['cumulative return'] = (TSM_stock_data['value'] - TSM_stock_data['value'].iat[0])/TSM_stock_data['value'].iat[0]
TSM_stock_data['volatility'] = TSM_stock_data['vwap'].rolling(len(TSM_stock_data),min_periods=1).std()
TSM_stock_data['sharpe'] = (TSM_stock_data['cumulative return'] - rf)/(TSM_stock_data['cumulative return'].rolling(len(TSM_stock_data),min_periods=1).std())
TSM_stock_data.fillna(0, inplace=True)

AXP_stock_data['value'] = (AXP_stock_data['vwap'] * AXP_shares).round(2)
AXP_stock_data['cumulative return'] = (AXP_stock_data['value'] - AXP_stock_data['value'].iat[0])/AXP_stock_data['value'].iat[0]
AXP_stock_data['volatility'] = AXP_stock_data['vwap'].rolling(len(AXP_stock_data),min_periods=1).std()
AXP_stock_data['sharpe'] = (AXP_stock_data['cumulative return'] - rf)/(AXP_stock_data['cumulative return'].rolling(len(AXP_stock_data),min_periods=1).std())
AXP_stock_data.fillna(0, inplace=True)

CELH_stock_data['value'] = (CELH_stock_data['vwap'] * CELH_shares).round(2)
CELH_stock_data['cumulative return'] = (CELH_stock_data['value'] - CELH_stock_data['value'].iat[0])/CELH_stock_data['value'].iat[0]
CELH_stock_data['volatility'] = CELH_stock_data['vwap'].rolling(len(CELH_stock_data),min_periods=1).std()
CELH_stock_data['sharpe'] = (CELH_stock_data['cumulative return'] - rf)/(CELH_stock_data['cumulative return'].rolling(len(CELH_stock_data),min_periods=1).std())
CELH_stock_data.fillna(0, inplace=True)

NVDA_stock_data['value'] = (NVDA_stock_data['vwap'] * NVDA_shares).round(2)
NVDA_stock_data['cumulative return'] = (NVDA_stock_data['value'] - NVDA_stock_data['value'].iat[0])/NVDA_stock_data['value'].iat[0]
NVDA_stock_data['volatility'] = NVDA_stock_data['vwap'].rolling(len(NVDA_stock_data),min_periods=1).std()
NVDA_stock_data['sharpe'] = (NVDA_stock_data['cumulative return'] - rf)/(NVDA_stock_data['cumulative return'].rolling(len(NVDA_stock_data),min_periods=1).std())
NVDA_stock_data.fillna(0, inplace=True)

USDJPY_forex_data['value'] = (USDJPY_forex_data['vwap'] * USDJPY_shares).round(2)
USDJPY_forex_data['cumulative return'] = (USDJPY_forex_data['value'] - USDJPY_forex_data['value'].iat[0])/USDJPY_forex_data['value'].iat[0]
USDJPY_forex_data['percent change'] = USDJPY_forex_data['value'].pct_change()
USDJPY_forex_data.fillna(0, inplace=True)
USDJPY_forex_data

,date,open,high,low,close,volume,vwap,transactions,value,cumulative return,percent change
0,2023-01-03,130.79,131.44,129.50,131.29,276808,130.47,276808,10000.00,0.000000,0.000000
1,2023-01-04,131.29,132.72,129.92,132.25,272444,131.18,272444,10054.42,0.005442,0.005442
2,2023-01-05,132.24,134.05,131.68,133.32,272141,132.88,272141,10184.72,0.018472,0.012959
3,2023-01-06,133.32,134.77,131.99,132.08,260547,133.56,260547,10236.84,0.023684,0.005117
4,2023-01-09,131.81,132.66,131.30,131.73,258141,131.93,258141,10111.90,0.011190,-0.012205
...,...,...,...,...,...,...,...,...,...,...,...
376,2024-07-03,161.49,162.00,160.76,161.59,100342,161.59,100342,12385.22,0.238522,0.000186
377,2024-07-05,161.37,161.39,160.32,160.72,127932,160.85,127932,12328.50,0.232850,-0.004580
378,2024-07-08,160.64,161.12,160.26,160.86,114113,160.72,114113,12318.54,0.231854,-0.000808
379,2024-07-09,160.86,161.51,160.74,161.33,106138,161.12,106138,12349.20,0.234920,0.002489


Performing portfolio aggregations

In [270]:
portfolio_aggs = pd.DataFrame(SPY_etf_data['date'])
portfolio_aggs['value'] = (
    SPY_etf_data['value'] +
    AMZN_stock_data['value'] +
    TSM_stock_data['value'] +
    AXP_stock_data['value'] +
    CELH_stock_data['value'] +
    NVDA_stock_data['value'] +
    USDJPY_forex_data['value']
)
portfolio_aggs['cumulative return'] = (portfolio_aggs['value'] - portfolio_aggs['value'].iat[0])/portfolio_aggs['value'].iat[0]

portfolio_aggs['annualized return'] = (
    (1 + portfolio_aggs['cumulative return']).pow(365/(portfolio_aggs['date'] - portfolio_aggs['date'].iat[0]).dt.days) - 1
)

less_than_1_year = (portfolio_aggs['date'] - portfolio_aggs['date'].iat[0]).dt.days < 365
# Global Investment Performance Standards dictate that returns of portfolios or composites for periods of less than one year may not be annualized
portfolio_aggs.loc[less_than_1_year, 'annualized return'] = 0

portfolio_aggs['volatility'] = portfolio_aggs['value'].rolling(len(portfolio_aggs),min_periods=2).std()

portfolio_aggs['sharpe'] = (portfolio_aggs['cumulative return'] - rf)/(portfolio_aggs['cumulative return'].rolling(len(portfolio_aggs),min_periods=1).std())
portfolio_aggs.fillna(0,inplace=True)
portfolio_aggs

,date,value,cumulative return,annualized return,volatility,sharpe
0,2023-01-03,100000.00,0.000000,0.000000,0.000000,0.000000
1,2023-01-04,100821.29,0.008213,0.000000,580.739728,-7.987589
2,2023-01-05,100104.03,0.001040,0.000000,447.176589,-11.977304
3,2023-01-06,101043.70,0.010437,0.000000,518.134235,-8.523467
4,2023-01-09,103385.28,0.033853,0.000000,1369.403536,-1.515054
...,...,...,...,...,...,...
376,2024-07-03,237825.46,1.378255,0.782659,38122.953031,3.472067
377,2024-07-05,240030.02,1.400300,0.789854,38317.841564,3.511942
378,2024-07-08,241367.70,1.413677,0.790769,38517.195700,3.528494
379,2024-07-09,242538.41,1.425384,0.794606,38719.994541,3.540249


Market Aggregations

In [257]:
NDX_data

,open,high,low,close,date
0,12085.67,12156.21,12006.04,12066.27,2023-02-22
1,11979.80,12018.32,11900.84,11969.65,2023-02-24
2,12106.79,12159.64,12034.61,12057.79,2023-02-27
3,12041.75,12146.52,12021.32,12042.12,2023-02-28
4,12026.72,12054.48,11906.58,11937.48,2023-03-01
...,...,...,...,...,...
344,20224.13,20406.99,20201.50,20391.97,2024-07-05
345,20393.89,20455.38,20363.37,20439.54,2024-07-08
346,20504.17,20543.90,20395.57,20453.02,2024-07-09
347,20533.27,20690.97,20479.94,20675.38,2024-07-10


In [278]:
# Denormalization
portfolio_db = pd.concat([
    portfolio_aggs,
    SPY_etf_data[['vwap', 'value', 'cumulative return', 'volatility', 'sharpe']],
    AMZN_stock_data[['vwap', 'value', 'cumulative return', 'volatility', 'sharpe']],
    TSM_stock_data[['vwap', 'value', 'cumulative return', 'volatility', 'sharpe']],
    AXP_stock_data[['vwap', 'value', 'cumulative return', 'volatility', 'sharpe']],
    CELH_stock_data[['vwap', 'value', 'cumulative return', 'volatility', 'sharpe']],
    NVDA_stock_data[['vwap', 'value', 'cumulative return', 'volatility', 'sharpe']],
    USDJPY_forex_data[['vwap', 'value', 'cumulative return', 'percent change']]
    ], join='outer', axis=1,
    )
portfolio_db.columns = ['date', 'value', 'cumulative return', 'annualized return', 'volatility', 'sharpe',
        'SPY vwap', 'SPY value', 'SPY cumulative return', 'SPY volatility', 'SPY sharpe',
        'AMZN vwap', 'AMZN value', 'AMZN cumulative return', 'AMZN volatility', 'AMZN sharpe',
        'TSM vwap', 'TSM value', 'TSM cumulative return', 'TSM volatility', 'TSM sharpe',
        'AXP vwap', 'AXP value', 'AXP cumulative return', 'AXP volatility', 'AXP sharpe',
        'CELH vwap', 'CELH value', 'CELH cumulative return', 'CELH volatility', 'CELH sharpe',
        'NVDA vwap', 'NVDA value', 'NVDA cumulative return', 'NVDA volatility', 'NVDA sharpe',
        'USDJPY vwap', 'USDJPY value', 'USDJPY cumulative return', 'USDJPY percent change']
portfolio_db

,date,value,cumulative return,annualized return,volatility,sharpe,SPY vwap,SPY value,SPY cumulative return,SPY volatility,...,CELH sharpe,NVDA vwap,NVDA value,NVDA cumulative return,NVDA volatility,NVDA sharpe,USDJPY vwap,USDJPY value,USDJPY cumulative return,USDJPY percent change
0,2023-01-03,100000.00,0.000000,0.000000,0.000000,0.000000,380.96,40000.00,0.000000,0.000000,...,0.000000,14.37,10000.00,0.000000,0.000000,0.000000,130.47,10000.00,0.000000,0.000000
1,2023-01-04,100821.29,0.008213,0.000000,580.739728,-7.987589,383.15,40229.95,0.005749,1.548564,...,-30.934366,14.62,10173.97,0.017397,0.176777,-3.024256,131.18,10054.42,0.005442,0.005442
2,2023-01-05,100104.03,0.001040,0.000000,447.176589,-11.977304,380.26,39926.50,-0.001837,1.507658,...,-10.749133,14.31,9958.25,-0.004175,0.164418,-5.137034,132.88,10184.72,0.018472,0.012959
3,2023-01-06,101043.70,0.010437,0.000000,518.134235,-8.523467,385.25,40450.44,0.011261,2.261128,...,-5.539236,14.60,10160.06,0.016006,0.158008,-3.509951,133.56,10236.84,0.023684,0.005117
4,2023-01-09,103385.28,0.033853,0.000000,1369.403536,-1.515054,390.36,40986.98,0.024675,4.060903,...,-5.109026,15.79,10988.17,0.098817,0.603796,1.052340,131.93,10111.90,0.011190,-0.012205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,2024-07-03,237825.46,1.378255,0.782659,38122.953031,3.472067,550.30,57780.34,0.444508,45.661683,...,1.269245,125.21,87132.92,7.713292,28.757503,3.827015,161.59,12385.22,0.238522,0.000186
377,2024-07-05,240030.02,1.400300,0.789854,38317.841564,3.511942,553.57,58123.69,0.453092,45.878605,...,1.269156,127.04,88406.40,7.840640,28.965116,3.862763,160.85,12328.50,0.232850,-0.004580
378,2024-07-08,241367.70,1.413677,0.790769,38517.195700,3.528494,555.17,58291.68,0.457292,46.100938,...,1.182584,128.59,89485.04,7.948504,29.179230,3.887539,160.72,12318.54,0.231854,-0.000808
379,2024-07-09,242538.41,1.425384,0.794606,38719.994541,3.540249,556.10,58389.33,0.459733,46.324822,...,1.076861,131.38,91426.58,8.142658,29.408277,3.952132,161.12,12349.20,0.234920,0.002489
